# Imports

In [ ]:
from PIL import Image
from bresenham import bresenham
from skimage.color import rgb2gray
from skimage.draw import line
import numpy as np
import matplotlib.pyplot as plt
import sys
import cv2
import dlib
import os

# Parameters

In [ ]:
# Parameters 
INPUT_IMAGE_PATH = "img/dali_huge.jpg"
OUTPUT_IMAGE_PATH = "out/" + INPUT_IMAGE_PATH[4:]
NAIL_DISTANCE = 35
ITERATIONS = 2000

MASK_BACKGROUND_COLOR = 100
MASK_FACE_COLOR = 120
MASK_FEATURES_COLOR = 150


# print("Parameters:")
# print("INPUT_IMAGE_PATH: " + INPUT_IMAGE_PATH)
# print("OUTPUT_IMAGE_PATH: " + OUTPUT_IMAGE_PATH)
# print("OUTPUT_IMAGE_DIMENSIONS: " + str(OUTPUT_IMAGE_DIMENSIONS))
# print("OUTPUT_IMAGE_RADIUS: " + str(OUTPUT_IMAGE_RADIUS))
# print("OUTPUT_IMAGE_CENTER_POSITION: " + str(OUTPUT_IMAGE_CENTER_POSITION))
# print("NAILS_AMOUNT: " + str(NAILS_AMOUNT))
# print("SKIP_NEIGHBOUR_NAILS: " + str(SKIP_NEIGHBOUR_NAILS))

In [ ]:


def display_image(image):
    display(Image.fromarray(image))

# 1:1 2560x2560 = 

# 3:2 2560x1440 = 1228800

# 4:3 960x720 = 2073600

# 5:4

# 7:5

# 16:9 2560x1440 = 5760000
# 9:16 1440x2560 = 5760000




In [ ]:
#
# Display 
#
def display_image(image):
    display(Image.fromarray(image))

def display_nails(nails, image):
    canvas = np.ones(image.shape, np.uint8) * 255
    for nail in nails:
        canvas[nail[0], nail[1]] = 0
    display_image(canvas)

def display_threads(lines, image):
    canvas = np.ones(image.shape, np.uint8) * 255
    for line in lines:
        canvas = cv2.line(canvas, (line[0][1], line[0][0]), (line[1][1], line[1][0]), 0, 1)
    display_image(canvas)

#
# Image
#
def get_input_image(): 
    img = cv2.imread(INPUT_IMAGE_PATH, 0)

    # Fit to aspect ration 

    return img 

def get_new_image(dimensions):
    return np.ones(dimensions, np.uint8) * 255

def get_input_image_edges():
    image = get_input_image()
    return cv2.Canny(image.copy(), 100, 200, apertureSize=3).astype(np.uint8)

def draw_line(image, line, color):
    return cv2.line(image, (line[0][1], line[0][0]), (line[1][1], line[1][0]), color, 1)

def draw_point(image, point, color):
    image[point[1], point[0]] = color
    return image

#
# Points
#
def generate_nail_positions(dimensions):
    nails = []

    r0, c0 = 0, 0
    r1, c1, = 0, dimensions[1] - 1
    r2, c2 = dimensions[0] - 1, dimensions[1] - 1
    r3, c3 = dimensions[0] - 1, 0

    for i in range(c0, c1, NAIL_DISTANCE):
        nails.append((c0, i))

    for i in range(r1, r2, NAIL_DISTANCE):
        nails.append((i, c1))

    for i in range(c2, c3, -NAIL_DISTANCE):
        nails.append((r2, i))
        
    for i in range(r3, r0, -NAIL_DISTANCE):
        nails.append((i, c3))

    return np.array(nails)


def is_on_same_edge(point1, point2, dimensions):
    if point1[0] == 0 and point2[0] == 0:
        return True
    if point1[0] == dimensions[0] - 1 and point2[0] == dimensions[0] - 1:
        return True
    if point1[1] == 0 and point2[1] == 0:
        return True
    if point1[1] == dimensions[1] - 1 and point2[1] == dimensions[1] - 1:
        return True
    return False

def generate_thread_positions(nails, dimensions):
    threads = []
    for i in range(0, len(nails)):
        for j in range(i, len(nails)):
            if not is_on_same_edge(nails[i], nails[j], dimensions):
                threads.append(np.array([nails[i], nails[j]]))
    return np.array(threads)

def get_other_end_of_thread(thread, nail):
    if np.array_equal(thread[0], nail):
        return thread[1]
    return thread[0]

def get_threads_from_nail(threads, nail):
    threads_from_nail = []
    for thread in threads:
        if np.array_equal(thread[0], nail) or np.array_equal(thread[1], nail):
            threads_from_nail.append(thread)
    return np.array(threads_from_nail)    

def generate_nail_to_threads_dictionary(nails, threads):
    nail_to_thread_dictionary = {}
    i = 0
    for nail in nails:
        print("Generating nail to thread dictionary: " + str(i) + "/" + str(len(nails)))
        nail_to_thread_dictionary[nail.tobytes()] = get_threads_from_nail(threads, nail)
        i += 1
    return nail_to_thread_dictionary

def generate_thread_to_points_dictionary(threads):
    thread_to_points_dictionary = {}
    i = 0
    for thread in threads:
        print("Thread " + str(i) + " of " + str(len(threads)))
        i += 1
        a, b = line(thread[0][0], thread[0][1], thread[1][0], thread[1][1])
        points = []
        for i in range(len(a)):
            points.append((a[i], b[i]))
        thread_to_points_dictionary[thread.tobytes()] = np.array(points)
    return thread_to_points_dictionary


#
# Test 
#
input_image = get_input_image()
input_image_edges = get_input_image_edges()
output_image = get_new_image(input_image.shape)

nails = generate_nail_positions(input_image.shape)
threads = generate_thread_positions(nails, input_image.shape)

nail_to_threads_dictionary = generate_nail_to_threads_dictionary(nails, threads)
thread_to_points_dictionary = generate_thread_to_points_dictionary(threads)

print("Input image")
display_image(input_image)

print("Input image edges")
display_image(input_image_edges)

print("Output image")
display_image(output_image)

print("All nails")
display_nails(nails, output_image)

print("First nail")
display_nails([nails[0]], output_image)

print("Nail from get_other_end_of_thread of first thread")
display_nails([get_other_end_of_thread(threads[0], nails[0])], output_image)

print("Threads from get_threads_from_nail")
display_threads(get_threads_from_nail(threads, nails[0]), output_image)

print("Threads from dictionary")
display_threads(nail_to_threads_dictionary[nails[0].tobytes()], output_image)

print("First thread points from dictionary")
display_nails(thread_to_points_dictionary[threads[0].tobytes()], output_image)

In [ ]:
# calculetes how much inputting this thread will decrease the quality of input image, hence improving the output image
def get_thread_panelty(thread, ttp, input_image, input_image_mask):
    a = input_image[tuple(ttp[thread.tobytes()].T)].sum()
    b = input_image_mask[tuple(ttp[thread.tobytes()].T)].sum()
    c = 0.15
    return (a + (c*b)) / len(ttp[thread.tobytes()])

    
def get_best_fitting_thread(threads, ttp, input_image, input_image_edges): # which thread cover the most black and the least white
    # print(threads)
    best_fitting_thread = threads[0]
    best_fitting_thread_penalty = 999999999
    for thread in threads:
        penalty = get_thread_panelty(thread, ttp, input_image, input_image_edges)
        # return thread
        if penalty < best_fitting_thread_penalty:
            best_fitting_thread = thread
            best_fitting_thread_penalty = penalty

    return best_fitting_thread



used_threads = []
current_nail = nails[0] # might change to start from a different nail
current_input_image = input_image.copy()

n = nails.copy()
t = threads.copy()
ntt = nail_to_threads_dictionary.copy()
ttp = thread_to_points_dictionary.copy()

for i in range(0, 15000):
    
    if i % 500 == 0: 
        print("Current nail #", i, ": ", str(current_nail))
        display_threads(used_threads, output_image)
        display_image(current_input_image)
        
    current_threads = ntt[current_nail.tobytes()]
    if len(current_threads) == 0:
        print("No threads for nail", current_nail)
        break
    selected_thread = get_best_fitting_thread(current_threads, ttp, current_input_image, input_image_edges)
    next_nail = get_other_end_of_thread(selected_thread, current_nail)

    used_threads.append(selected_thread)


    # Delete selected thread from ntt
    ntt[current_nail.tobytes()] = np.array([a for a, skip in zip(ntt[current_nail.tobytes()], 
                                    [np.allclose(a, selected_thread) for a in ntt[current_nail.tobytes()]]) if not skip])

    # Delete selected thread from next_nail
    ntt[next_nail.tobytes()] = np.array([a for a, skip in zip(ntt[next_nail.tobytes()], 
                                    [np.allclose(a, selected_thread) for a in ntt[next_nail.tobytes()]]) if not skip])
    
    if len(ntt[current_nail.tobytes()]) == 1:
        for thread in ntt[current_nail.tobytes()]:
            print(thread)
            other_nail = get_other_end_of_thread(thread, current_nail)
            print(other_nail)
            ntt[current_nail.tobytes()] = np.array([a for a, skip in zip(ntt[current_nail.tobytes()], 
                                    [np.allclose(a, thread) for a in ntt[current_nail.tobytes()]]) if not skip])
            ntt[other_nail.tobytes()] = np.array([a for a, skip in zip(ntt[other_nail.tobytes()], 
                                    [np.allclose(a, thread) for a in ntt[other_nail.tobytes()]]) if not skip])


    draw_line(current_input_image, selected_thread, 255)
    current_nail = next_nail

current_input_image.save('dalitest.png')
display_threads(used_threads, output_image)


# Own algorithm

In [ ]:

# # Start at random nail 
# # Get all unused threds from that nail
# # Put through fitness function
# # Select best thread
# # Add to used threads
# # Go to next nail
# # Repeat until fitness function returns positive values
# # If no positive values, undo last nail and choose second best thread (parametrize how many step to go back)
# # Repeat until no improvement can be made

# # Save image

# input_image = get_input_image()
# input_image_mask = get_input_image_mask(input_image)
# output_image = generate_new_image(OUTPUT_IMAGE_DIMENSIONS)
# nails = generate_nail_positions(OUTPUT_IMAGE_RADIUS, OUTPUT_IMAGE_CENTER_POSITION, NAILS_AMOUNT)
# threads = generate_thread_positions(nails)
# # display(nails)
# # display(nails[0])

# # display(input_image)
# # display(threads)
# # display(threads[0])
# # display_points(nails, output_image)

# ntt = generate_nail_to_threads_dictionary(nails, threads)
# ttp = generate_thread_to_points_dictionary(threads)
# # display_threads(ntt[nails[0]], output_image)

# # display(ntt)
# # display(ntt[nails[0]])

# # display(ttp[threads[0]])